<h1 style="color:rgb(0,120,170)">Artificial Intelligence in Life Sciences</h1>
<h2 style="color:rgb(0,120,170)">QSAR and model evaluation</h2>

<b>Authors:</b> Rumetshofer, Renz, Schimunek <br>
<b>Date:</b> 24-03-2022

This file is part of the "Artificial Intelligence in Life Sciences" lecture material.
The following copyright statement applies to all code within this file.

<b>Copyright statement:</b><br>
This material, no matter whether in printed or electronic form, may be used for personal and non-commercial educational
use only. Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed
or in electronic form, requires explicit prior acceptance of the authors.

In [38]:
import os
import pandas as pd
import numpy as np
import copy

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

from rdkit import RDLogger  
RDLogger.DisableLog('rdApp.*') 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering

from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

print(rdkit.__version__)

2022.09.4


<h1 style="color:rgb(0,120,170)">The Tox21 dataset</h1>

The Tox21 dataset comprises roughly 13,000 compounds (12,060 training samples, 647 test samples) tested for 12 different toxicological assays (active/inactive). The label matrix contains a lot of missing labels denoted as <i>NA</i>.

More information about the dataset can be found here: https://tripod.nih.gov/tox21/challenge/

Here we use a version of the dataset where the samples have been clustered and then assigned to 5 different folds.

In [6]:
# Preprocessed Tox21 dataset with pre-assigned clusters
data = pd.read_csv("resources/tox21_cleaned.csv",index_col=0).reset_index(drop=True)
data

,Smiles,Split,NR-AhR,NR-AR,NR-AR-LBD,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,cluster_folds,cluster_split
0,C[n+]1c2cc(N)ccc2cc2ccc(N)cc21.Nc1ccc2cc3ccc(N...,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4,test
1,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1,train
2,CO[C@H]1CC(O[C@H]2C[C@H]([C@H]3O[C@](C)(O)[C@H...,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1,train
3,CN(C)c1ccc(C(=C2C=CC(=[N+](C)C)C=C2)c2ccccc2)c...,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2,train
4,CC(=O)O.CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=...,train,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12693,OCCCS,test,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,val
12694,OCCN1CN(CCO)CN(CCO)C1,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,test
12695,Cn1[nH]nnc1=S,test,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,2,train
12696,OC(Cc1ccccc1Cl)(Cn1[nH]cnc1=S)C1(Cl)CC1,test,0.0,0.0,0.0,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,1.0,2,train


<h2 style="color:rgb(0,120,170)">Data preprocessing</h2>

In order to use the dataset for training a model we replace the missing values with `-1`.

In [39]:
# Select labels, replace NaNs, convert to numpy array
y = data[data.columns[2:-2]].fillna(-1)
y = y.to_numpy()
y

array([[-1., -1., -1., ...,  0., -1., -1.],
       [-1., -1., -1., ...,  0., -1., -1.],
       [-1., -1., -1., ...,  0., -1., -1.],
       ...,
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ..., -1., -1.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [40]:
y.shape

(12698, 12)

Next, we calculate Morgan fingerprints from the Smiles string for each sample.

In [41]:
# Initialize variables
fp_length = 1024
fps = np.zeros((len(data), fp_length))

# Calculate Morgan fingerprints and convert to numpy array
for i, smiles in enumerate(tqdm(data['Smiles'])):
    mol = Chem.MolFromSmiles(smiles)
    fp_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=fp_length)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_vec, arr)
    fps[i] = arr

100%|██████████| 12698/12698 [00:03<00:00, 4004.46it/s]


In [44]:
fps

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

<h1 style="color:rgb(0,120,170)">Train model on random split</h1>

We can now train a random forest model using the Morgan fingerprints as input. Here we don't use the clustering information but train the model on the standard train/test split where samples were randomly assigned to the respective set.

In [45]:
# Function returning the training and test sets
def split_data(test_fold, X, y, folds):
    test_mask = test_fold==folds.values
    X_test = X[test_mask]
    y_test = y[test_mask]

    train_mask = test_fold!=folds.values
    X_train = X[train_mask]
    y_train = y[train_mask]

    return X_train, X_test, y_train, y_test

# Train a random forest model for each task on the supplied training set and return predictions for the test set
def train_rf(X_train, y_train, X_test):
    seed = 1234
    n_tasks = y_train.shape[1]
    y_hats_proba = np.empty((X_test.shape[0], n_tasks))
    y_hats_class = np.empty_like(y_hats_proba)
    
    # Train RF per task
    for j in tqdm(range(n_tasks)):
        rf_model = RandomForestClassifier(n_estimators=100, random_state=seed)
        # Mask out unknown samples
        idx = (y_train[:, j] != (-1)) # True where label is 0 or 1, False for unknowns
        # Train model
        rf_model.fit(X_train[idx], y_train[idx, j])
        # Predict class probabilities (select only values for positiv class with index 1)
        y_hats_proba[:, j] = rf_model.predict_proba(X_test)[:, 1]
        # Predict class 
        y_hats_class[:, j] = rf_model.predict(X_test)
    return y_hats_proba, y_hats_class 

In [46]:
# Split data into training and test set using the random split from the dataset
X_train, X_test, y_train, y_test = split_data("test", fps, y, data['Split'])
# Train a random forest model and get predictions for the test set
y_hats_proba, y_hats_class = train_rf(X_train, y_train, X_test)

100%|██████████| 12/12 [00:33<00:00,  2.76s/it]


We can look at the predicted probabilities and classes. Since the model doesn't know which values of the test set are actually measured and which are missing we get a prediction for each sample.

In [47]:
pd.DataFrame(y_hats_proba)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.198333,0.480,0.360,0.127,0.380000,0.206000,0.07,0.420000,0.130000,0.1380,0.25,0.190000
1,0.225000,0.020,0.040,0.090,0.170000,0.050000,0.07,0.240000,0.080000,0.0700,0.20,0.210000
2,0.365000,0.040,0.010,0.060,0.090000,0.040000,0.12,0.336667,0.096667,0.0400,0.17,0.136667
3,0.160000,0.000,0.010,0.090,0.120000,0.036667,0.01,0.175000,0.150000,0.0425,0.09,0.060000
4,0.063333,0.005,0.235,0.060,0.116667,0.060000,0.07,0.195000,0.140000,0.0600,0.15,0.140000
...,...,...,...,...,...,...,...,...,...,...,...,...
640,0.000000,0.000,0.000,0.000,0.010000,0.000000,0.00,0.020000,0.000000,0.0000,0.00,0.000000
641,0.000000,0.020,0.004,0.000,0.020000,0.010000,0.00,0.080000,0.000000,0.0150,0.01,0.000000
642,0.106667,0.015,0.000,0.020,0.010000,0.010000,0.01,0.040000,0.000000,0.0750,0.04,0.050000
643,0.202000,0.030,0.040,0.080,0.182500,0.040000,0.00,0.280000,0.050000,0.1600,0.28,0.060000


In [49]:
pd.DataFrame(y_hats_class)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We can also look at the predicted number of samples for each class.

In [51]:
unique, counts = np.unique(y_hats_class, return_counts=True)
dict(zip(unique, counts))

{0.0: 7610, 1.0: 130}

<h1 style="color:rgb(0,120,170)">Metrics</h1>

To determine the quality of the model we look at several metrics. When calculating metrics we need to remove predictions for missing values as there's no way to measure the quality of these predictions.

<h2 style="color:rgb(0,120,170)">Confusion Matrix, Precision, Recall, F1-score</h2>

Lets look at these metrics (or methods) for the first task.

In [52]:
task = 0
# Mask out unknown samples
idx = (y_test[:, task] != (-1))

### Confusion Matrix

In [16]:
cm = confusion_matrix(y_test[idx,task], y_hats_class[idx,task])
cm

array([[527,  10],
       [ 64,   7]])

In [17]:
# True Negatives, False Positives, False Negatives, True Positives
cm.ravel()

array([527,  10,  64,   7])

### Precision, Recall and F1-Score

- The **precision** is the ratio $\frac{TP}{TP + FP}$ where TP is the number of true positives and FP the number of false positives. The precision is intuitively the ability of the classifier to not label negative samples as positive.

- The **recall** is the ratio $\frac{TP}{TP + FN}$ where TP is the number of true positives and FN the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

- The **F1-score** can be interpreted as a weighted harmonic mean of the precision and recall.

In [18]:
print(classification_report(y_test[idx,task],y_hats_class[idx,task], target_names=["class 0", "class 1"]))

              precision    recall  f1-score   support

     class 0       0.89      0.98      0.93       537
     class 1       0.41      0.10      0.16        71

    accuracy                           0.88       608
   macro avg       0.65      0.54      0.55       608
weighted avg       0.84      0.88      0.84       608



<h2 style="color:rgb(0,120,170)">Area under the ROC curve (AUC)</h2>

Next, we calculate the AUC for each task and the mean over all tasks.

In [53]:
def calc_masked_AUC_per_task(prediction, target):
    auc_per_task = []
    for j in range(target.shape[1]):
        y_score = prediction[:, j]
        y_true = target[:, j]
        # Mask out unknown samples
        idx = (y_true != (-1))
        # Calculate AUC per task
        auc_per_task.append(roc_auc_score(y_true[idx], y_score[idx]))
    return auc_per_task

In [54]:
# Calculate AUC per task
auc_per_task = calc_masked_AUC_per_task(y_hats_proba, y_test)
auc_per_task

[0.8783539224171847,
 0.7007113821138211,
 0.7090968586387434,
 0.7082481254260395,
 0.7375483870967742,
 0.6972366148531952,
 0.7961030904579292,
 0.7089149297368668,
 0.7057391868712624,
 0.7477894053703253,
 0.9019775826372528,
 0.7709494773519163]

In [55]:
np.mean(auc_per_task)

0.7552224135809426

<h1 style="color:rgb(0,120,170)">Cluster Cross-Validation</h1>

The previous model was trained with samples randomly assigned to the training and test sets. However, if we want to know how well our model generalizes to future data it might be a better idea to assign the training and test samples based on structural similarity. If we cluster the samples and assign all samples of some clusters to the training set and all samples of the other clusters to the test set we avoid that very similar samples are in the training and test sets.

In [22]:
# We have 5 different cluster folds
data['cluster_folds'].unique()

array([4, 1, 2, 3, 0])

In [56]:
# We can select a list of test-folds here, to save time we only select one
test_folds = [0]

# For each test_fold we train a model on the remaining folds and calculate the AUC on the selected test fold
for test_fold in test_folds:
    # Split data
    X_train, X_test, y_train, y_test = split_data(test_fold, fps, y, data['cluster_folds'])
    y_hats_proba, y_hats_class = train_rf(X_train, y_train, X_test)

    # Calculate mean AUC
    auc_per_task = calc_masked_AUC_per_task(y_hats_proba, y_test)
    print(np.mean(auc_per_task))

100%|██████████| 12/12 [00:24<00:00,  2.08s/it]

0.7419875728081738
